# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [34]:
import pandas as pd

In [35]:
# Import the data
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [36]:
# Total number of people who survived/didn't survive
df.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [55]:
# Create dummy variables
relevant_columns = ['Pclass', 'SibSp', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], drop_first=True, dtype=float)

dummy_dataframe.shape

(891, 6)

In [56]:
dummy_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass        891 non-null int64
SibSp         891 non-null int64
Survived      891 non-null int64
Sex_male      891 non-null float64
Embarked_Q    891 non-null float64
Embarked_S    891 non-null float64
dtypes: float64(3), int64(3)
memory usage: 41.8 KB


Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [47]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(714, 7)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [57]:
# Split the data into X and y
y = dummy_dataframe['Survived']
X = dummy_dataframe.drop(dummy_dataframe['Survived'])

In [58]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [59]:
X.head()

,Pclass,SibSp,Survived,Sex_male,Embarked_Q,Embarked_S
2,3,0,1,0.0,0.0,1.0
3,1,1,1,0.0,0.0,1.0
4,3,0,0,1.0,0.0,1.0
5,3,0,0,1.0,1.0,0.0
6,1,0,0,1.0,0.0,1.0


## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [62]:
# I got an error using the stats model and google said to try this...
y = list(y)

In [64]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm

# Create intercept term required for sm.Logit, see documentation for more information
X = sm.tools.add_constant(X)

# Fit model
logit_model = sm.Logit(y, X)

# Get results of the fit
result = logit_model.fit()

ValueError: endog and exog matrices are different sizes

## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [65]:
# Summary table
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      709
Method:                           MLE   Df Model:                            4
Date:                Wed, 19 Feb 2020   Pseudo R-squ.:                  0.3399
Time:                        14:51:14   Log-Likelihood:                -318.36
converged:                       True   LL-Null:                       -482.26
                                        LLR p-value:                 1.089e-69
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6008      0.543     10.306      0.000       4.536       6.666
Pclass        -1.3174      0.141     -9.350      0.000      -1.594      -1.041
Age           -0.0444      0.008     -5.442      0.000      -0.060      -0.028
SibSp         -0.3761      0.121     -3.106      0.002      -0.613      -0.139
Sex_male      -2.6235      0.215    -12.229      0.000      -3.044      -2.203
==============================================================================
"""

In [32]:
# Your comments here
# Pseudo R-squared is 0.339 which isn't very good!

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [33]:
# Your code here
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Sex', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], drop_first=True, dtype=float)

dummy_dataframe = dummy_dataframe.dropna()

y = dummy_dataframe['Survived']
X = dummy_dataframe.drop(columns=['Survived'], axis=1)

X = sm.tools.add_constant(X)
logit_model = sm.Logit(y, X)
result = logit_model.fit()

result.summary()

Optimization terminated successfully.
         Current function value: 0.445882
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      709
Method:                           MLE   Df Model:                            4
Date:                Wed, 19 Feb 2020   Pseudo R-squ.:                  0.3399
Time:                        14:46:11   Log-Likelihood:                -318.36
converged:                       True   LL-Null:                       -482.26
                                        LLR p-value:                 1.089e-69
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6008      0.543     10.306      0.000       4.536       6.666
Pclass        -1.3174      0.141     -9.350      0.000      -1.594      -1.041
Age           -0.0444      0.008     -5.442      0.000      -0.060      -0.028
SibSp         -0.3761      0.121     -3.106      0.002      -0.613      -0.139
Sex_male      -2.6235      0.215    -12.229      0.000      -3.044      -2.203
==============================================================================
"""

In [ ]:
# Your comments here

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!